In [49]:
import os
import json
import pandas as pd

json_files = ['acer.json', 'apple.json', 'asus.json', 'honor.json', 'hp.json', 
              'huawei.json', 'lenovo.json', 'msi.json', 'nokia.json', 
              'samsung.json', 'xiaomi.json']

all_data = []

for filename in os.listdir():
    if filename in json_files:  
        with open(filename, 'r', encoding='utf-8') as file:
            data = json.load(file)
            all_data.append(data) 

In [51]:
def persian_to_english_numbers(s):
    numbers = {'۰': '0', '۱': '1', '۲': '2', '۳': '3', '۴': '4',
               '۵': '5', '۶': '6', '۷': '7', '۸': '8', '۹': '9'}
    if isinstance(s, str): 
        for persian_digit, english_digit in numbers.items():
            s = s.replace(persian_digit, english_digit)
    return s


if all_data:
    dfs = [pd.DataFrame(data) for data in all_data]
    combined_df = pd.concat(dfs, ignore_index=True)
    print(f"Number of all data: {len(combined_df)}")
    combined_df['price'] = combined_df['price'].apply(persian_to_english_numbers)
    combined_df['price'] = combined_df['price'].str.replace(r'[٪,]', '', regex=True)
    combined_df['price'] = pd.to_numeric(combined_df['price'], errors='coerce')  # تبدیل به نوع عددی
    print(combined_df['price'].head(5))
else:
    print("No data has been loaded")


Number of all data: 1901
0    7.237900e+08
1    1.931890e+09
2    1.228999e+09
3    1.221000e+09
4    1.321099e+09
Name: price, dtype: float64


In [55]:
print(combined_df.head())

  id                                               link  \
0  1  https://www.digikala.com/product/dkp-11019312/...   
1  2  https://www.digikala.com/product/dkp-11579612/...   
2  3  https://www.digikala.com/product/dkp-9833679/%...   
3  4  https://www.digikala.com/product/dkp-11274902/...   
4  5  https://www.digikala.com/product/dkp-10400100/...   

                                                name         price rate  \
0  لپ تاپ 15.6 اینچی ایسر مدل Aspire 3 A315-58G-i...  7.237900e+08  ۴.۸   
1  لپ تاپ 15.6 اینچی ایسر مدل Aspire 7 A715-42G-R2YB  1.931890e+09  ۴.۷   
2  لپ تاپ 15.6 اینچی ایسر مدل Aspire 3 A315-58G-7...  1.228999e+09    ه   
3  لپ تاپ 15.6 اینچی ایسر مدل Aspire A315-23-R3PE...  1.221000e+09    ه   
4  لپ تاپ 15.6 اینچی ایسر مدل Aspire 3 A315-58G-3...  1.321099e+09    ۵   

   freeSend       status image  
0      True  ارسال امروز   NaN  
1      True  ارسال امروز   NaN  
2      True                NaN  
3      True                NaN  
4      True                Na

In [35]:
# تبدیل اعداد فارسی در ستون 'rate' به انگلیسی
combined_df['rate'] = combined_df['rate'].apply(persian_to_english_numbers)
combined_df['rate'] = pd.to_numeric(combined_df['rate'], errors='coerce')  # تبدیل به نوع عددی
print(combined_df[['rate']].head(5))


   rate
0   4.8
1   4.7
2   NaN
3   NaN
4   5.0


In [37]:
# محاسبه تعداد مقادیر NaN در ستون‌های 'rate' و 'price'
nan_rate_count = combined_df['rate'].isna().sum()
nan_price_count = combined_df['price'].isna().sum()

print(f"Number of NaN values in 'rate': {nan_rate_count}")
print(f"Number of NaN values in 'price': {nan_price_count}")


Number of NaN values in 'rate': 505
Number of NaN values in 'price': 1052


In [59]:
#   محاسبه میانگین قیمت برای هر مدل
mean_prices = combined_df.groupby('name')['price'].mean()

def fill_missing_price(row):
    if pd.isna(row['price']):
        return mean_prices[row['name']] 
    return row['price']

combined_df['price'] = combined_df.apply(fill_missing_price, axis=1)
print(combined_df[['name', 'price']].head(5))


                                                name         price
0  لپ تاپ 15.6 اینچی ایسر مدل Aspire 3 A315-58G-i...  7.237900e+08
1  لپ تاپ 15.6 اینچی ایسر مدل Aspire 7 A715-42G-R2YB  1.931890e+09
2  لپ تاپ 15.6 اینچی ایسر مدل Aspire 3 A315-58G-7...  1.228999e+09
3  لپ تاپ 15.6 اینچی ایسر مدل Aspire A315-23-R3PE...  1.221000e+09
4  لپ تاپ 15.6 اینچی ایسر مدل Aspire 3 A315-58G-3...  1.321099e+09
